In [1]:
import numpy as np
import math


In [38]:
# Need to compute the time it takes to do something given a jerk, max_accel, vax_v

cj = ConstantJerk(None, 5500, 2500, 15000)
cj()


[0, 0.45454545454545453, 0.45454545454545453, 6.454545454545454]

In [39]:
cj = ConstantJerk(2000, 5500, 2500, 15000)
cj()


[1.118033988749895, 1.118033988749895, 2.23606797749979, 8.23606797749979]

In [36]:






class ConstantJerk:
    """Initially from https://github.com/mdhom/py_constant_jerk/blob/main/constantJerk.py
    """
    def __init__(self, jerk_max, acc_max, v_max, s):
        self.jerk_max = jerk_max
        self.acc_max = acc_max
        self.v_max = v_max

        self.trajectory_instance_case = self.__get_trajectory_instance_case(s)
        tj, ta, tv = self.__calculate_times(s)
        self.t1 = tj
        self.t2 = ta
        self.t3 = ta + tj
        self.t4 = tv
        self.t5 = tv + tj
        self.t6 = tv + ta
        self.t7 = tv + tj + ta

    def __call__(self):
        return [self.t1, self.t2, self.t3, self.t7]

    def __get_trajectory_instance_case(self, s: float):
        # Case 1: a_max reached, v_max reached and constant for a time
        #          (s = 100,   j = 2000, a = 500,  vMax = 120)
        # Case 2: a_max not reached, v_max not reached
        #          (s = 15000, j = 2000, a = 5500, vMax = 20500)
        # Case 3: a_max not reached, v_max reached and constant for a time
        #          (s = 15000, j = 2000, a = 5500, vMax = 2500)
        # Case 4: a_max reached, v_max not reached
        #          (s = 57,    j = 2000, a = 500,  vMax = 120)
        # Case 5: a_max reached and constant for a time, v_max reached and constant for a time
        #          (s = 15000, j = 2000, a = 500,  vMax = 2500)
        # Case 6: a_max reached and constant for a time, v_max not reached
        #          (s = 15000, j = 2000, a = 500,  vMax = 20500)

        if self.jerk_max is None:
            v_a = 0
            s_a = 0
        else:
            v_a = self.acc_max * self.acc_max / self.jerk_max
            s_a = 2 * self.acc_max * self.acc_max * self.acc_max / (self.jerk_max * self.jerk_max)
        s_v = 0.0
        if self.jerk_max is not None:
            if self.v_max * self.jerk_max < self.acc_max * self.acc_max:
                s_v = self.v_max * 2 * math.sqrt(self.v_max / self.jerk_max)
            else:
                s_v = self.v_max * (self.v_max / self.acc_max + self.acc_max / self.jerk_max)

        if self.v_max <= v_a and s > s_a:
            return 1
        elif self.v_max > v_a and s < s_a:
            return 2
        elif self.v_max < v_a and s_a > s > s_v:
            return 3
        elif self.v_max < v_a and s < s_a and s < s_v:
            return 4
        elif self.v_max > v_a and s > s_a and s >= s_v:
            return 5
        elif self.v_max > v_a and s_a < s < s_v:
            return 6
        else:
            return None

    def __calculate_times(self, s: float):
        if self.trajectory_instance_case == 1 or self.trajectory_instance_case == 3:
            if self.jerk_max is None:
                tj = 0
            else:
                tj = math.sqrt(self.v_max / self.jerk_max)
            ta = tj
            tv = s / self.v_max
            return tj, ta, tv
        elif self.trajectory_instance_case == 2 or self.trajectory_instance_case == 4:
            if self.jerk_max is None:
                tj = 0
            else:
                tj = (s / (2 * self.jerk_max))**(1./3.)
            ta = tj
            tv = 2 * tj
            return tj, ta, tv
        elif self.trajectory_instance_case == 5:
            if self.jerk_max is None:
                tj = 0
            else:
                tj = self.acc_max / self.jerk_max
            ta = self.v_max / self.acc_max
            tv = s / self.v_max
            return tj, ta, tv
        elif self.trajectory_instance_case == 6:
            if self.jerk_max is None:
                tj = 0
                ta = 0
            else:
                tj = self.acc_max / self.jerk_max
                ta = 0.5 * (math.sqrt(
                    (4 * s * self.jerk_max * self.jerk_max + self.acc_max * self.acc_max * self.acc_max) / (
                                self.acc_max * self.jerk_max * self.jerk_max)) - self.acc_max / self.jerk_max)
            tv = ta + tj
            return tj, ta, tv
        else:
            raise Exception("TrajectoryInstance must be between 1 and 6")

    def get_status(self, t: float):
        if t <= self.t1:
            return self.__get_status1(t)
        else:
            j1, a1, v1, s1 = self.__get_status1(self.t1)
            if t <= self.t2:
                return self.__get_status2(t, a1, v1, s1)
            else:
                j2, a2, v2, s2 = self.__get_status2(self.t2, a1, v1, s1)
                if t <= self.t3:
                    return self.__get_status3(t, a2, v2, s2)
                else:
                    j3, a3, v3, s3 = self.__get_status3(self.t3, a2, v2, s2)
                    if t <= self.t4:
                        return self.__get_status4(t, v3, s3)
                    else:
                        j4, a4, v4, s4 = self.__get_status4(self.t4, v3, s3)
                        if t <= self.t5:
                            return self.__get_status5(t, v4, s4)
                        else:
                            j5, a5, v5, s5 = self.__get_status5(self.t5, v4, s4)
                            if t <= self.t6:
                                return self.__get_status6(t, a5, v5, s5)
                            else:
                                j6, a6, v6, s6 = self.__get_status6(self.t6, a5, v5, s5)
                                return self.__get_status7(min(t, self.t7), a6, v6, s6)

    def __get_status1(self, t: float):
        if self.jerk_max is None:
            j = None
            a = self.acc_max
            v = self.acc_max * t
            s = 0.5 * self.acc_max * t * t
        else:
            j = self.jerk_max
            a = self.jerk_max * t
            v = 0.5 * self.jerk_max * t * t
            s = self.jerk_max / 6 * t * t * t
        return j, a, v, s

    def __get_status2(self, t: float, a1: float, v1: float, s1: float):
        j = 0
        a = a1
        v = v1 + a1 * (t - self.t1)
        s = s1 + v1 * (t - self.t1) + 0.5 * a1 * (t - self.t1) * (t - self.t1)
        return j, a, v, s

    def __get_status3(self, t: float, a2: float, v2: float, s2: float):
        t_phase = t - self.t2
        t_phase2 = t_phase * t_phase
        t_phase3 = t_phase2 * t_phase
        
        j = -self.jerk_max
        a = a2 - self.jerk_max * t_phase
        v = v2 + a2 * t_phase + 0.5 * -self.jerk_max * t_phase2
        s = s2 + v2 * t_phase + 0.5 * a2 * t_phase2 + -self.jerk_max / 6 * t_phase3
        return j, a, v, s

    def __get_status4(self, t: float, v3: float, s3: float):
        t_phase = t - self.t3
        j = 0
        a = 0
        v = v3
        s = s3 + v3 * t_phase
        return j, a, v, s

    def __get_status5(self, t: float, v4: float, s4: float):
        t_phase = t - self.t4
        t_phase2 = t_phase * t_phase
        t_phase3 = t_phase2 * t_phase
        j = -self.jerk_max
        a = -self.jerk_max * t_phase
        v = v4 + 0.5 * -self.jerk_max * t_phase2
        s = s4 + v4 * t_phase + -self.jerk_max / 6 * t_phase3
        return j, a, v, s

    def __get_status6(self, t: float, a5: float, v5: float, s5: float):
        t_phase = t - self.t5
        t_phase2 = t_phase * t_phase
        j = 0
        a = a5
        v = v5 - self.acc_max * t_phase
        s = s5 + v5 * t_phase + 0.5 * a5 * t_phase2
        return j, a, v, s

    def __get_status7(self, t: float, a6: float, v6: float, s6: float):
        t_phase = t - self.t6
        t_phase2 = t_phase * t_phase
        t_phase3 = t_phase2 * t_phase
        j = self.jerk_max
        a = a6 + self.jerk_max * t_phase
        v = v6 + a6 * t_phase + 0.5 * self.jerk_max * t_phase2
        s = s6 + v6 * t_phase + 0.5 * a6 * t_phase2 + self.jerk_max / 6 * t_phase3
        return j, a, v, s

In [30]:
cj = ConstantJerk(2, 10, 10, 1)

In [31]:
cj()

[0.6299605249474366, 2.5198420997897464]

In [26]:
cj.get_status(50.)

(2, 4.440892098500626e-16, 3.885780586188048e-16, 1.0000000000000004)

In [28]:
cj.t7

2.5198420997897464

In [1]:
from jerk import jerk_time
import numpy as np

In [2]:
jerk_time(np.array(3.5), 7., 7., None), jerk_time(np.array(3.5), 7., 7., 1e6)

(array([1.41421356]), array([1.41422056]))

In [3]:
jerk_time(np.array(3.5), 7., 7., 4)

array([3.03658897])

In [4]:
jerk_time(np.array(3.5), 3.5, 3.5, None), jerk_time(np.array(3.5), 3.5*.7, 3.5*.7, 6)

(array([2.]), array([2.83690476]))

In [5]:
# Case 1: a_max reached, v_max reached and constant for a time
        #          (s = 100,   j = 2000, a = 500,  vMax = 120)
        # Case 2: a_max not reached, v_max not reached
        #          (s = 15000, j = 2000, a = 5500, vMax = 20500)
        # Case 3: a_max not reached, v_max reached and constant for a time
        #          (s = 15000, j = 2000, a = 5500, vMax = 2500)
        # Case 4: a_max reached, v_max not reached
        #          (s = 57,    j = 2000, a = 500,  vMax = 120)
        # Case 5: a_max reached and constant for a time, v_max reached and constant for a time
        #          (s = 15000, j = 2000, a = 500,  vMax = 2500)
        # Case 6: a_max reached and constant for a time, v_max not reached
        #          (s = 15000, j = 2000, a = 500,  vMax = 20500)

In [5]:
jerk_time(np.arange(10), 3.5, 3.5, None)

array([0.        , 1.06904497, 1.51185789, 1.8516402 , 2.14285714,
       2.42857143, 2.71428571, 3.        , 3.28571429, 3.57142857])

In [6]:
jerk_time(np.arange(10), 3.5, 3.5, 6)

array([0.        , 1.74716093, 2.20128483, 2.52468575, 2.79957031,
       3.0439356 , 3.29761905, 3.58333333, 3.86904762, 4.1547619 ])